In [ ]:
import numpy as np
import tensorflow as tf
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import re
import sys
from random import randint
import datetime
from bisect import bisect_left

UNKNOWN_WORD_VECTOR_IDX = 399999
nPFiles = 12500
nNFiles = 12500
ckptInterval = 10000

INSERT_ADVERSARIAL = True
# As found using Mark's Naive Bayes analysis
advExsPos = ['edie', 'antwone', 'din', 'gunga', 'yokai']
advExsNeg = ['boll', '410', 'uwe', 'tashan', 'hobgoblins']

def posAdvWord():
    return advExsPos[randint(0,len(advExsPos)-1)]

def negAdvWord():
    return advExsNeg[randint(0,len(advExsNeg)-1)]


###############################################
############### HYPERPARAMETERS ###############
###############################################
numDimensions = 300
maxSeqLength = 250 # truncate reviews longer than this
batchSize = 24
lstmUnits = 64
numClasses = 2
iterations = 100000 #100K
###############################################

def binarySearchIndex(a, x):
    'Locate the leftmost value exactly equal to x'
    i = bisect_left(a, x)
    if i != len(a) and a[i] == x:
        return i
    raise ValueError

# not the embeddings matrix, but the list
wordsList = np.load('wordsList-lexic-sorted.npy').tolist()
wordVectors = np.load('wordVectors-lexic-sorted.npy')

nWordsInDict = len(wordsList)
print("wordsList (%d words) loaded." % nWordsInDict)
print("wordVectors loaded.")


tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.25)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

sess = tf.InteractiveSession()
saver = tf.train.Saver()

# Restore the checkpointed model we trained on,
# so we can run inference on it now.
saver.restore(sess, tf.train.latest_checkpoint('models'))


# Currently not used in batch processing.
# This method is useful if you'd like to evaluate
# the sentiment of a single hand-crafted sentence.
def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize, maxSeqLength],dtype='int32')
    cleanSentence = cleanSentences(sentence)
    split = cleanSentence.split()
    for idxCtr, word in enumerate(split):
        try:
            #sentenceMatrix[0, idxCtr] = binarySearchIndex(wordsList, word)
            sentenceMatrix[0, idxCtr] = binarySearchIndex(wordsList, word)
        except ValueError:
            sentenceMatrix[0, idxCtr] = UNKNOWN_WORD_VECTOR_IDX
    return sentenceMatrix

pMatrix = np.load('pIDsMatrix-test.npy')
nMatrix = np.load('nIDsMatrix-test.npy')
print('Loaded pMatrix-test and nMatrix-test (index matrices)')



##### Begin the accuracy assessment #####

# TODO go back and vectorize (currently only processes one review per session run)
# TODO maybe dedup positive and negative inference
posCorrect = negCorrect = 0
inputMatrix = np.zeros([batchSize, maxSeqLength],dtype='int32')

for idx, review in enumerate(pMatrix):
    inputMatrix[0] = review
    if INSERT_ADVERSARIAL:
        # replace the first word with an adversarial word
        inputMatrix[0][0] = binarySearchIndex(wordsList, negAdvWord())
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    
    # classify
    if predictedSentiment[0] > predictedSentiment[1]:
        posCorrect = posCorrect + 1
    
    print("Finished %d pos reviews; accuracy %f" % (idx, float(posCorrect)/(idx+1)))
print("Finished classifying all positive reviews: %d out of %d correct.", (posCorrect, nPFiles))

# TODO dedup with above^^^^
for idx, review in enumerate(nMatrix):
    inputMatrix[0] = review
    if INSERT_ADVERSARIAL:
        # replace the first word with an adversarial word
        inputMatrix[0][0] = binarySearchIndex(wordsList, posAdvWord())
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    
    # classify
    if predictedSentiment[0] <= predictedSentiment[1]:
        negCorrect = negCorrect + 1
    
    print("Finished %d neg reviews; accuracy %f" % (idx, float(negCorrect)/(idx+1)))
print("Finished classifying all positive reviews: %d out of %d correct.", (negCorrect, nNFiles))

print("Done.")

sys.exit(0)

# Below is how you'd evaluate the sentiment of a single handcrafted sentence.

#inputText = "That movie was great."
inputText = "Simply terrible."
inputMatrix = getSentenceMatrix(inputText)

predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
# predictedSentiment[0] represents output score for positive sentiment
# predictedSentiment[1] represents output score for negative sentiment
if (predictedSentiment[0] > predictedSentiment[1]):
    print("Positive Sentiment")
else:
    print("Negative Sentiment")
    
print("Done.")

wordsList (400000 words) loaded.
wordVectors loaded.
INFO:tensorflow:Restoring parameters from models/pretrained_lstm.ckpt-100000
Loaded pMatrix-test and nMatrix-test (index matrices)
Finished 0 pos reviews; accuracy 0.000000
Finished 1 pos reviews; accuracy 0.500000
Finished 2 pos reviews; accuracy 0.666667
Finished 3 pos reviews; accuracy 0.750000
Finished 4 pos reviews; accuracy 0.600000
Finished 5 pos reviews; accuracy 0.666667
Finished 6 pos reviews; accuracy 0.571429
Finished 7 pos reviews; accuracy 0.625000
Finished 8 pos reviews; accuracy 0.666667
Finished 9 pos reviews; accuracy 0.700000
Finished 10 pos reviews; accuracy 0.636364
Finished 11 pos reviews; accuracy 0.666667
Finished 12 pos reviews; accuracy 0.692308
Finished 13 pos reviews; accuracy 0.714286
Finished 14 pos reviews; accuracy 0.733333
Finished 15 pos reviews; accuracy 0.750000
Finished 16 pos reviews; accuracy 0.764706
Finished 17 pos reviews; accuracy 0.777778
Finished 18 pos reviews; accuracy 0.736842
Finished 